In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

# TODO: auto sim cutoff needed
decomp = cluster_model.Decomposer(model, ds, labs)
decomp.load()

Creating decomposer with parameter hash a9ba5d753fd502dc8bbf7b6ccce8ebfe0455dfd8
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Getting cluster scores for lattice
Set all initial to 0
Set predictions for layer 0
Set predictions for layer 1
Set predictions for layer 2
Set predictions for layer 3
Set predictions for layer 4
Set predictions for layer 5
Set all probs with predictions
Scoring layer 0
COEFF STUFF 1.0 0.0 (512, 512) -0.393107 0.99582356
Scoring layer 1
COEFF STUFF 1.0 0.0 (512, 512) -0.17596376 0.99558663
Scoring layer 2
COEFF STUFF 1.0 3.8e-44 (512, 512) -0.18728897 0.98405635
Scoring layer 3
COEFF STUFF 1.0 0.0 (512, 512) -0.31588098 0.99695206
Scoring layer 4
COEFF STUFF 1.0 4.72e-43 (512, 512) -0.09775802 0.9802551
CLUSTER SCORES [array([[ 6.9218270e-02,  7.2610470e-05, -2.0057986e-04, ...,
        -1.5130841e-04,  9.3730392e-05,  9.8355979e-02],
       [ 8.2

In [3]:
decomp.lattice_scores[0].shape

(512, 512)

In [4]:
from typing import List
import utils
importlib.reload(utils)
modified_lattice = decomp.lattice_scores.copy()

# TODO: have function which does something like fixing from the a specific neuron?
# Ie have the sub-lattice with only **related** neurons

def cutoff_lattice(lattice: List[List[List[float]]], related_cutoff = 1):
    print(lattice[0].sum())
    r = [(layer > related_cutoff) * layer for layer in lattice]
    print(r[0].sum())
    return r

def create_sublattice(lattice, layer: int, idx_in_layer: int, related_cutoff = 10):
	with_cutoff = cutoff_lattice(lattice, related_cutoff)

LAST_FEAT = 20
modified_lattice = cutoff_lattice(decomp.lattice_scores, related_cutoff=15)

# utils.top_k_dag_paths_dynamic(modified_lattice, k=10_000)

1486.7596
0.0


## Isolate Specific Neurons

In [5]:
import numpy as np

def get_weighting_for_layer(layer: int, n_layers: int, weight_decay=0.95):
    r = np.ones(n_layers)
    r[layer] = 1
    G = weight_decay
    for i in range(layer):
        r[i] = G ** (layer - i)
    for i in range(layer + 1, n_layers):
        # Decrease by Gx per layer
        r[i] = G ** (i - layer)
    return r
    

In [6]:
decomp.lattice_scores[0][:, 13]

array([ 1.01307577e-07,  4.07907268e-04, -7.30547890e-06, -3.68692126e-04,
        2.10330007e-03,  2.13061809e-03,  1.12687948e-03,  5.86531532e-04,
        1.05061044e-03, -2.03796924e-04,  1.61544871e-04,  2.65995841e-02,
       -2.76010367e-04,  6.65347371e-03,  2.49926792e-03, -4.07663756e-05,
        1.70308544e-04,  4.98488953e-04,  2.59113889e-02,  4.17617662e-03,
        1.90744596e-03,  7.30644260e-03,  1.50852511e-02, -1.27557607e-03,
        3.35112010e-04, -1.28975051e-04,  1.30763063e-02,  3.02586495e-03,
        2.73487950e-03,  8.86857219e-04,  1.93173275e-03,  5.52231038e-04,
        1.49040471e-03, -9.80107434e-05, -3.33595177e-04, -4.02648118e-04,
        2.67690537e-03, -1.48257503e-04,  8.46439871e-05,  8.80618521e-04,
        2.30196791e-04,  2.03641161e-04, -1.89656930e-04,  1.40469440e-03,
       -1.49690444e-04,  2.90222056e-02,  3.77765857e-02,  1.56903889e-06,
        3.91227566e-03,  1.07315707e-03, -3.66270076e-04,  1.87260367e-03,
        3.87994712e-03,  

In [7]:
importlib.reload(utils)
# TODO: restrict lattice, change picture instead of starting elsewhere...
# ~~Use source~~ node
# TODO: ~~SOMEHOW LAYER 2 is where stuff happens~~
cutoff = 0.3
scores_cutoff = [m * m > cutoff for m in modified_lattice]
paths = utils.top_k_dag_paths(decomp.lattice_scores, layer=1, neuron=13, k=50)
paths

([105, 13, 300, 222, 222, 180], 3.9362623413085935)
([105, 13, 59, 423, 368, 418], 3.8499123413085936)
([105, 13, 505, 73, 484, 397], 3.784272341308594)
([177, 13, 300, 222, 222, 180], 3.760342341308594)
([401, 13, 300, 222, 222, 180], 3.7499823413085935)
([157, 13, 300, 222, 222, 180], 3.7475523413085936)
([70, 13, 300, 222, 222, 180], 3.739722341308594)
([105, 13, 194, 245, 92, 70], 3.736082341308594)
([330, 13, 300, 222, 222, 180], 3.7254723413085937)
([75, 13, 300, 222, 222, 180], 3.7210723413085938)
([105, 13, 505, 73, 484, 388], 3.7179423413085937)
([105, 13, 262, 245, 92, 70], 3.7130423413085936)
([186, 13, 300, 222, 222, 180], 3.7008523413085936)
([105, 13, 460, 112, 87, 402], 3.677842341308594)
([177, 13, 59, 423, 368, 418], 3.6739923413085935)
([431, 13, 300, 222, 222, 180], 3.6721623413085935)
([321, 13, 300, 222, 222, 180], 3.6667523413085936)
([401, 13, 59, 423, 368, 418], 3.6636323413085936)
([381, 13, 300, 222, 222, 180], 3.661632341308594)
([157, 13, 59, 423, 368, 418],

[([105, 13, 300, 222, 222, 180], 3.9362623413085935),
 ([105, 13, 59, 423, 368, 418], 3.8499123413085936),
 ([105, 13, 505, 73, 484, 397], 3.784272341308594),
 ([177, 13, 300, 222, 222, 180], 3.760342341308594),
 ([401, 13, 300, 222, 222, 180], 3.7499823413085935),
 ([157, 13, 300, 222, 222, 180], 3.7475523413085936),
 ([70, 13, 300, 222, 222, 180], 3.739722341308594),
 ([105, 13, 194, 245, 92, 70], 3.736082341308594),
 ([330, 13, 300, 222, 222, 180], 3.7254723413085937),
 ([75, 13, 300, 222, 222, 180], 3.7210723413085938),
 ([105, 13, 505, 73, 484, 388], 3.7179423413085937),
 ([105, 13, 262, 245, 92, 70], 3.7130423413085936),
 ([186, 13, 300, 222, 222, 180], 3.7008523413085936),
 ([105, 13, 460, 112, 87, 402], 3.677842341308594),
 ([177, 13, 59, 423, 368, 418], 3.6739923413085935),
 ([431, 13, 300, 222, 222, 180], 3.6721623413085935),
 ([321, 13, 300, 222, 222, 180], 3.6667523413085936),
 ([401, 13, 59, 423, 368, 418], 3.6636323413085936),
 ([381, 13, 300, 222, 222, 180], 3.6616323413

## Setup for scoring and visualization

In [8]:
import math
import numpy as np
from IPython.core.display import display, HTML
from circuitsvis.utils.render import render

/tmp/ipykernel_201349/672954307.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Try different scoring method

In [9]:
# # TODO: scoring and testing the paths

# importlib.reload(cluster_model)

# # score_path = [69, 2, 493, 289, 511, 48]
# # TODO: embeds to here...
# to_score = ds
# to_score_adj = [s[:300] for s in to_score][:300]
# to_score = to_score_adj
# importlib.reload(cluster_model)
# # TODO: should we work over log everywhere?
# _, top_ds, scores = cluster_model.score_for_neuron(decomp, to_score_adj, LAYER, score_path, top_n=200, weighting_per_layer=get_weighting_for_layer(LAYER, N_BLOCKS))

# def get_renderable_scores(scored_t: List[str], scores: List


# scores_per_token_set = np.array([max(s) for s in scores])
# top_args = np.argsort(scores_per_token_set)[::-1]

# # scores_per_token_set = [max(s) for s in scores]
# # top_args = [s.argmax() for s in scores]

# to_score_top = [to_score[i] for i in top_ds]

# tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
#     'input_ids']] for d in to_score_top]
# tokens_reord = [tokens[i] for i in top_args]
# scores_reord = [scores[i] for i in top_args]

# # TODO: WHAT IS HAPPENING WITH NAN?
# act_simp = [[[[math.exp(10 * tok)]]
#              for tok in s] for s in scores_reord]
# # TODO sep fun
# html = render(
#     "TextNeuronActivations",
#     tokens=tokens_reord,
#     activations=act_simp,
#     firstDimensionName="Layer",
#     secondDimensionName="Neuron",
#     firstDimensionLabels=None,
#     secondDimensionLabels=None
# )
# display(HTML(str(html)))

## Try more path like method

In [11]:
"""
score_path = [13, 13, 391, 57, 57, 57]... relates to laws/ licensing
"""

# TODO: SIOMETHING WRONG HERE WITH THE PATHS
import numpy as np
import metric
importlib.reload(cluster_model)
importlib.reload(utils)
importlib.reload(metric)
score_path = [105, 13, 460, 112, 87, 402]
LAYER = 1
# score_path = [0, 0, 69, 51, 195, 99]
# score_path = [69, 1, 107, 289, 97, 508]

"""
Paths which seem to have meaning:

"""

weighting_per_layer = get_weighting_for_layer(LAYER, N_BLOCKS, weight_decay=0.9)
# weighting_per_layer[0] = 0
# weighting_per_layer[1] = 0
print(weighting_per_layer)
to_score = [d[:300] for d in ds][:200]
scores = decomp.score(
    to_score,
    score_path=score_path,
    # TODO: how do we weight???
    weighting_per_layer=weighting_per_layer,
    use_log_scores=True
)

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

# TODO: we have the weights here

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]

[0.9    1.     0.9    0.81   0.729  0.6561]
GOT EMBEDS (6, 14946, 512)
(14946, 6, 512) [105, 13, 300, 222, 222, 180]
COS INNER PRODUCT [[-0.00352425 -0.0063639   0.00170703 ... -0.02794641 -0.00156283
   0.01749546]
 [ 0.01267044  0.01311709 -0.001498   ... -0.00615564  0.0254414
  -0.02682352]
 [-0.01353815  0.02944623 -0.00316041 ... -0.01483284 -0.04469972
   0.00961424]
 ...
 [-0.00352425 -0.0063639   0.00170703 ... -0.02794641 -0.00156283
   0.01749546]
 [ 0.04069512 -0.00490523 -0.03433636 ... -0.00581758  0.03933388
  -0.03385713]
 [ 0.02950817  0.08213786  0.03552478 ...  0.05745233  0.01962953
   0.00488949]]
COS INNER PRODUCT [[-0.02045425  0.00266846  0.00490958 ... -0.00377521 -0.01135762
  -0.01453211]
 [-0.01565942  0.00537063 -0.0021275  ...  0.03440934  0.0273174
  -0.01997694]
 [-0.01810233 -0.02127166 -0.02639428 ...  0.04813257 -0.01950232
  -0.03124876]
 ...
 [-0.02045425  0.00266846  0.00490958 ... -0.00377521 -0.01135762
  -0.01453211]
 [-0.01927935 -0.00254888 -0

In [ ]:
import math
from IPython.core.display import display, HTML
import math
import numpy as np

# TODO: WHAT IS HAPPENING WITH NAN?
# act_simp = [[[[(math.exp(tok * 10))]]
#              for tok in s] for s in scores_reord]
act_simp = [[[[math.exp(tok + 30)]]
             for tok in s] for s in scores_reord]
# TODO sep fun
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

/tmp/ipykernel_198232/3814091935.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Look at distributions of scores

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get distribution of lattice scores

LAYER_START = 4
cutoff = 0.25

# ax = sns.heatmap(decomp.lattice_scores[LAYER_START])
plt.imshow((decomp.lattice_scores[LAYER_START] > cutoff), cmap='hot', interpolation='nearest')
plt.show()

plt.plot((decomp.lattice_scores[3] > cutoff).sum(axis=1))